# 语音识别目标追踪

### 导入头文件

In [1]:
import cv2 as cv
import threading
import random
from time import sleep
import ipywidgets as widgets
from IPython.display import display
from color_follow import color_follow
from face_follow import face_follow

### 初始化机械臂位置

In [2]:
import Arm_Lib
Arm = Arm_Lib.Arm_Device()
joints_0 = [90, 135, 20, 25, 90, 30]
Arm.Arm_serial_servo_write6_array(joints_0, 1000)

### 创建实例,初始化参数

In [3]:
follow = color_follow()
follow2 = face_follow()
# 初始化模式
model = 'General'
# 初始化HSV_learning值
HSV_learning = ()
# 初始化HSV值
color_hsv = {"red": ((170, 124, 134), (229, 242, 255)),
             "green": ((54, 113, 64), (75, 255, 255)),
             "blue": ((102, 150, 124), (124, 253, 255)),
             "yellow": ((22, 125, 130), (47, 255, 255))}
# 设置随机颜色
color = [[random.randint(0, 255) for _ in range(3)] for _ in range(255)]
# HSV参数路径
HSV_path="/home/yahboom/dofbot_ws/src/dofbot_color_follow/HSV_config.txt"
try: read_HSV(HSV_path,color_hsv)
except Exception: print("Read HSV_config Error !!!")

Read HSV_config Error !!!


### 创建控件

In [4]:
button_layout = widgets.Layout(width='200px', height='100px', align_self='center')
# 输出控件
output = widgets.Output()
# 颜色追踪
color_follow = widgets.Button(description='color_follow', button_style='success', layout=button_layout)
# 选择颜色
choose_color = widgets.ToggleButtons(options=['red', 'green', 'blue', 'yellow'], button_style='success',
             tooltips=['Description of slow', 'Description of regular', 'Description of fast'])
# 取消追踪
follow_cancel = widgets.Button(description='follow_cancel', button_style='danger', layout=button_layout)
# 学习颜色
learning_color = widgets.Button(description='learning_color', button_style='primary', layout=button_layout)
# 学习颜色追踪
learning_follow = widgets.Button(description='learning_follow', button_style='success', layout=button_layout)
# 退出
exit_button = widgets.Button(description='Exit', button_style='danger', layout=button_layout)
# 图像控件
imgbox = widgets.Image(format='jpg', height=480, width=640, layout=widgets.Layout(align_self='auto'))
# 垂直布局
img_box = widgets.VBox([imgbox, choose_color], layout=widgets.Layout(align_self='auto'))
# 垂直布局
Slider_box = widgets.VBox([color_follow, learning_color, learning_follow,follow_cancel,exit_button],
                          layout=widgets.Layout(align_self='auto'))
# 水平布局
controls_box = widgets.HBox([img_box, Slider_box], layout=widgets.Layout(align_self='auto'))
# ['auto', 'flex-start', 'flex-end', 'center', 'baseline', 'stretch', 'inherit', 'initial', 'unset']

### 模式切换

In [5]:
def color_follow_Callback(value):
    global model
    model = 'color_follow'
def learning_color_Callback(value):
    global model
    model = 'learning_color'
def learning_follow_Callback(value):
    global model
    model = 'learning_follow'
def follow_cancel_Callback(value):
    global model
    model = 'General'
def exit_button_Callback(value):
    global model
    model = 'Exit'
color_follow.on_click(color_follow_Callback)
learning_color.on_click(learning_color_Callback)
learning_follow.on_click(learning_follow_Callback)
follow_cancel.on_click(follow_cancel_Callback)
exit_button.on_click(exit_button_Callback)

In [6]:
Arm.Arm_ask_speech(9)
sleep(0.001)

### 主程序

In [7]:
def camera():
    global HSV_learning,model
    # 打开摄像头
    capture = cv.VideoCapture(0)
    capture.set(3, 640)
    capture.set(4, 480)
    capture.set(5, 30)  #设置帧率
    # 当摄像头正常打开的情况下循环执行
    while capture.isOpened():
        try:
            result = Arm.Arm_serial_speech_read(1)
            sleep(0.01)
            # 读取相机的每一帧
            _, img = capture.read()
            # 统一图像大小
            img = cv.resize(img, (640, 480))
            
            if result == 37:
                choose_color.value = 'red'
                model = 'color_follow'
              
                # Speech_text("好的，开始追踪红色",EncodingFormat_Type["GB2312"])
               
            elif result == 38:
                choose_color.value = 'yellow'
                model = 'color_follow'
               
                # Speech_text("好的，开始追踪黄色",EncodingFormat_Type["GB2312"])
               
            elif result == 39:
                choose_color.value = 'green'
                model = 'color_follow'
              
                # Speech_text("好的，开始追踪绿色",EncodingFormat_Type["GB2312"])
              
            elif result == 40:
                choose_color.value = 'blue'
                model = 'color_follow'
                
                # Speech_text("好的，开始追踪蓝色",EncodingFormat_Type["GB2312"])
                 
            elif result == 41:
                model = 'follow2'
                result = 255
                # ("好的，开始追踪人脸")
                 
            elif result == 42:
                model = 'General'
               
                # Speech_text("好的，取消追踪",EncodingFormat_Type["GB2312"])
               
            
            if model == 'color_follow':
                img = follow.follow_function(img, color_hsv[choose_color.value])
                # 添加文字
                cv.putText(img, choose_color.value, (int(img.shape[0] / 2), 50), cv.FONT_HERSHEY_SIMPLEX, 2, color[random.randint(0, 254)], 2)
            if model == 'follow2':
                img = follow2.follow_function(img)
            if model == 'learning_color':
                img,HSV_learning = follow.get_hsv(img)
            if model == 'learning_follow' :
                if len(HSV_learning)!=0:
                    print(HSV_learning)
                    img = follow.learning_follow(img, HSV_learning)
                    # 添加文字
                    cv.putText(img,'LeColor', (240, 50), cv.FONT_HERSHEY_SIMPLEX, 1, color[random.randint(0, 254)], 1)
            if model == 'Exit':
                cv.destroyAllWindows()
                capture.release()
                break
            imgbox.value = cv.imencode('.jpg', img)[1].tobytes()
        except KeyboardInterrupt:capture.release()

### 启动

In [8]:
display(controls_box,output)
threading.Thread(target=camera, ).start()

Output()

[ WARN:0] global ../modules/videoio/src/cap_gstreamer.cpp (935) open OpenCV | GStreamer warning: Cannot query video position: status=0, value=-1, duration=-1


-66.93182373046876
-2.4345744739879263
-84.41942249644887
9.612696876210615
89.5
-48.36363636363637
88.52272727272728
-51.23760330578512
48.638429752066116
-38.97614575507138
142.6944027047333
-46.63419506864285
162.35858206406667
-58.53492682442209
186.12350746036972
-78.77590243858384
193.71577340548816
-85.36599113078034
217.2696157641353
-92.1469082846164
242.36541961492142
-89.94517348041967
267.9650381468111
-88.24501577094725
290.0195489224374
-96.81772870644977
313.2063226293125
-105.86979351876818
337.3823929663012
-113.98816304716077
354.4438539060274
-117.04437845883281
349.5403503550934
-116.64039804171209
326.2309409413722
-123.7400361856102
311.4528128128521
-131.81727601687365
287.6548011648047
-134.86975236517034
259.1049819240732
-132.7154320331973
222.895907447643
-132.36049382119975
198.37690067705844
-136.64640852920002
183.19335460700535
-138.9451280481091
172.31303223700047
-141.74501164073718
155.77845747609095
-144.9995465127943
130.752587043281
-146.40904968298

##### 